In [1]:
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.feature_extraction.text import CountVectorizer





Feature Engineering of the Point data in Monuments. the aim is to enable the removal of the classification labels and the creation of meaningful features to allow ML training

In [2]:
# Read in the data sites nw features


In [14]:
sites_nw_clean.nunique()


OID_          9485
OBJECTID      9485
ENTITY_ID     9485
CLASS_CODE      82
SMRS          9485
COUNTY           4
TOWNLAND      3758
COUNTY_ID        4
ZONE_ID       7631
ITM_E         8729
ITM_N         8647
LATITUDE      9170
LONGITUDE     9212
MONUMENT_T      82
FIRST_EDIT     307
LATEST_EDI     826
WEBSITE_LI    9485
WEB_NOTES     8763
geometry      9249
dtype: int64

In [17]:
sites_nw_clean['MONUMENT_T'].unique()


array(['Cairn - boundary cairn', 'Bawn', 'Ringfort - rath', 'Church',
       'Cairn - unclassified', 'Castle - unclassified', 'Standing stone',
       'Megalithic structure', 'Crannog', 'Souterrain',
       'Megalithic tomb - portal tomb', 'Ritual site - holy well',
       'Ringfort - cashel', 'Cross', 'Ringfort - unclassified',
       'Barrow - unclassified', 'Megalithic tomb - unclassified',
       'Ring-ditch', 'Stone row', 'Megalithic tomb - court tomb', 'Mound',
       'Megalithic tomb - wedge tomb', 'Boundary mound', 'Ford',
       'Castle - tower house', 'Hilltop enclosure', 'Stone circle',
       'Promontory fort - inland', 'Megalithic tomb - passage tomb',
       'Round tower', 'Ecclesiastical enclosure',
       'Ecclesiastical residence', 'Cross-inscribed stone',
       'Ceremonial enclosure', 'Enclosure - large enclosure',
       'Cross - Market cross', 'Inscribed stone', 'Concentric enclosure',
       'Settlement cluster', 'Linear earthwork',
       "Ritual site - holy/sain

In [4]:
sites_nw_clean['WEB_NOTES']

0       Marked only on the 1910 ed. of the OS 6-inch m...
1       Marked only on the 1910 ed. of the OS 6-inch m...
2       Marked only on the 1910 ed. of the OS 6-inch m...
3       Marked only on the 1910 ed. of the OS 6-inch m...
4       Davies (1948a, 115) recorded the site of a pla...
                              ...                        
9480    In pasture, on the level top of a low hill/dru...
9481    In good coastal grassland, 1km S of Ballysadar...
9482    In a rath (SL019-264001-), close to the NE arc...
9483    In rough, rush-grown upland pasture, located o...
9484    Reported by Jean-Charles Caillere; visible on ...
Name: WEB_NOTES, Length: 9485, dtype: object

In [5]:
# Initialize a CountVectorizer object
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1,2))

# Fit and transform the WEB_NOTES column
X = vectorizer.fit_transform(sites_nw_clean['WEB_NOTES'].values.astype('U'))

# Get the feature names (words or phrases)
feature_names = vectorizer.get_feature_names_out()

# Sum up the counts of each feature (word or phrase)
sum_words = X.sum(axis=0)

# Create a dictionary of words/phrases and their counts
words_freq = [(word, sum_words[0, idx]) for word, idx in zip(feature_names, list(range(X.shape[1])))]

# Sort the words/phrases by their counts
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

# Print the 25 most common words/phrases
for word, freq in words_freq[:25]:
    print(word, freq)

bank 4731
area 4719
int 3927
circular 3906
diam 3599
wth 3577
5m 2904
circular area 2825
enclosed 2644
earthen 2627
pasture 2615
raised 2475
fosse 2452
situated 2363
stone 2224
earthen bank 2208
slope 2132
facing 2094
se 2059
ne 2015
os 1975
sw 1969
low 1956
nw 1936
area int 1857


In [12]:
# Search for a specific word
search_word = 'rectangular'
for word, freq in words_freq:
    if word == search_word:
        print(freq)

# Create a list of words/phrases to search for, and store it in a variable.

search_words = ['single banked','double banked','triple banked','univallate','bivallate', 'trivallate', 'rectangular', 'square', 'round', 'oval', 'octagonal', 'hexagonal', 'polygonal', 'circular', 'subrectangular', 'suboval`.`', 'subcircular', 'quadrilateral', 'width', 'length', 'diameter', 'height', 'depth', 'size', 'dimension', 'shape', 'area', 'circumference', 'perimeter', 'diameter', 'radius', 'diagonal', 'width', 'length', 'height', 'depth', 'thickness', 'size', 'dimension', 'shape', 'area', 'volume', 'circumference', 'perimeter', 'diameter', 'radius', 'diagonal', 'width', 'length', 'height', 'depth', 'thickness', 'size', 'dimension', 'shape', 'area', 'volume', 'circumference', 'perimeter', 'diameter', 'radius', 'diagonal', 'width', 'length', 'height', 'depth', 'size', 'dimension', 'shape', 'area', 'volume', 'circumference', 'perimeter', 'diameter', 'radius', 'diagonal', 'width', 'length', 'height', 'depth', 'thickness', 'size', 'dimension', 'shape', 'area', 'volume', 'circumference', 'perimeter', 'diameter', 'radius', 'diagonal', 'width', 'length', 'height', 'depth', 'thickness', 'size', 'dimension', 'shape', 'area', 'volume', 'circumference', 'perimeter', 'diameter', 'radius', 'diagonal', 'diam', 'entrance']

# Create a dictionary to store the counts of each word/phrase
search_words_freq = {}

# Loop through each word/phrase in the list
for search_word in search_words:
    # Loop through each word/phrase in the words_freq list
    for word, freq in words_freq:
        # If the word/phrase in the words_freq list matches the search word/phrase
        if word == search_word:
            # Add the word/phrase and its count to the dictionary
            search_words_freq[search_word] = freq

# Print the dictionary
print(search_words_freq)





299
{'single banked': 5, 'double banked': 2, 'bivallate': 7, 'trivallate': 1, 'rectangular': 299, 'square': 43, 'round': 134, 'oval': 932, 'octagonal': 3, 'polygonal': 7, 'circular': 3906, 'subrectangular': 131, 'subcircular': 496, 'width': 96, 'length': 23, 'diameter': 112, 'height': 209, 'depth': 9, 'size': 14, 'dimension': 9, 'shape': 11, 'area': 4719, 'circumference': 6, 'perimeter': 191, 'radius': 2, 'thickness': 12, 'volume': 109, 'diam': 3599, 'entrance': 1082}


NameError: name 'sample_entries' is not defined

In [ ]:
# Assuming `monuments` is a GeoDataFrame with your monuments data
# Extract the x and y coordinates from the geometry
points = np.array([monument.geometry.coords[0] for monument in monuments])

# Compute Voronoi tesselation
vor = Voronoi(points)

# Plot
voronoi_plot_2d(vor)
plt.show()